# AgricGPT - Fine-tuning with CROP-benchmark Evaluation

- **QLoRA** fine-tuning on AI4Agr/CROP-dataset
- **Evaluation** using official CROP-benchmark (accuracy metrics)
- **Model card** with benchmark results pushed to HuggingFace

In [1]:
%pip install -q torch transformers datasets peft bitsandbytes accelerate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 44.5 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
login()

## Configuration

In [23]:
import torch, math, json, re, os
from collections import defaultdict

# Model
MODEL_NAME = "microsoft/phi-2"
OUTPUT_DIR = "./agri_model_results"

# HuggingFace
HF_MODEL_NAME = "agricgpt-v1-phi2"
PUSH_TO_HUB = True
SAVE_STEPS = 100

# Data
DATASET_SIZE = 5000
VALIDATION_SPLIT = 0.1
MAX_SEQ_LENGTH = 512
BENCHMARK_SAMPLE_SIZE = 500

# LoRA
LORA_R, LORA_ALPHA, LORA_DROPOUT = 16, 32, 0.05
TARGET_MODULES = ["fc1", "fc2", "q_proj", "k_proj", "v_proj", "dense"]

# Training
NUM_EPOCHS, BATCH_SIZE = 3, 2
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 2e-4
LOGGING_STEPS, EVAL_STEPS = 10, 50

assert torch.cuda.is_available(), "GPU required!"
print(f"GPU: {torch.cuda.get_device_name(0)}")

GPU: NVIDIA A100-SXM4-40GB


## Load Model

In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig, pipeline

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, quantization_config=bnb_config, trust_remote_code=True, device_map={"":0})
model.config.use_cache = False
print(f"Loaded: {MODEL_NAME}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded: microsoft/phi-2


## Load CROP-benchmark (English)

In [25]:
from datasets import load_dataset

benchmark = load_dataset("AI4Agr/CROP-benchmark", split="test")

def is_english(text):
    if not text: return False
    return sum(1 for c in text if ord(c) < 128) / len(text) > 0.7

english_benchmark = benchmark.filter(lambda x: is_english(x.get("Question", "")))
print(f"English questions: {len(english_benchmark)}")

if BENCHMARK_SAMPLE_SIZE:
    english_benchmark = english_benchmark.shuffle(seed=42).select(range(min(BENCHMARK_SAMPLE_SIZE, len(english_benchmark))))
    print(f"Using {len(english_benchmark)} for evaluation")

English questions: 2522


## MCQ Evaluation Functions

In [26]:
def format_mcq_prompt(q, opts):
    return f"""### Instruction:
Answer the following agricultural question by selecting the correct option (A, B, C, or D).

Question: {q}

A) {opts['A']}
B) {opts['B']}
C) {opts['C']}
D) {opts['D']}

Reply with only the letter of the correct answer.

### Response:
"""

def extract_answer(response):
    response = response.strip().upper()
    match = re.search(r'\b([ABCD])\b', response)
    return match.group(1) if match else (response[0] if response and response[0] in 'ABCD' else None)

def evaluate_mcq(pipe, gen_config, data):
    correct, total = 0, 0
    by_level = defaultdict(lambda: {"correct": 0, "total": 0})

    for item in data:
        q = item.get("Question", "")
        opts = {"A": item.get("Option A", ""), "B": item.get("Option B", ""), "C": item.get("Option C", ""), "D": item.get("Option D", "")}
        ans = item.get("Answer", "").strip().upper()
        level = item.get("Level", "Unknown")

        if not q or not ans: continue

        torch.manual_seed(42)
        result = pipe(format_mcq_prompt(q, opts), generation_config=gen_config)
        pred = extract_answer(result[0]['generated_text'].split("### Response:")[-1])

        if pred == ans:
            correct += 1
            by_level[level]["correct"] += 1
        total += 1
        by_level[level]["total"] += 1

    return {"accuracy": correct/total if total else 0, "correct": correct, "total": total, "by_level": dict(by_level)}

print("Evaluation functions ready")

Evaluation functions ready


## Base Model Evaluation (BEFORE Training)

In [8]:
base_pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
gen_config = GenerationConfig(max_new_tokens=10, do_sample=False, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)

print("Evaluating base model on CROP-benchmark...")
base_results = evaluate_mcq(base_pipe, gen_config, english_benchmark)
print(f"\n📊 Base Model Accuracy: {base_results['accuracy']:.2%} ({base_results['correct']}/{base_results['total']})")

Device set to use cuda:0


Evaluating base model on CROP-benchmark...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



📊 Base Model Accuracy: 73.20% (366/500)


## Prepare Training Data

In [14]:
dataset = load_dataset("AI4Agr/CROP-dataset", data_files="**/*_en/**/*.json", split="train")
if DATASET_SIZE: dataset = dataset.select(range(min(DATASET_SIZE, len(dataset))))

def format_instruction(s):
    return {"text": f"### Instruction:\n{s['instruction']}\n\n### Response:\n{s['output']}{tokenizer.eos_token}"}

dataset = dataset.map(format_instruction)
dataset = dataset.train_test_split(test_size=VALIDATION_SPLIT, seed=42)
train_ds, eval_ds = dataset["train"], dataset["test"]

def tokenize(examples):
    return tokenizer(examples["text"], truncation=True, max_length=MAX_SEQ_LENGTH, padding="max_length")

tok_train = train_ds.map(tokenize, batched=True, remove_columns=train_ds.column_names)
tok_eval = eval_ds.map(tokenize, batched=True, remove_columns=eval_ds.column_names)
print(f"Train: {len(tok_train)}, Eval: {len(tok_eval)}")

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Train: 4500, Eval: 500


## Configure LoRA

In [10]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(r=LORA_R, lora_alpha=LORA_ALPHA, target_modules=TARGET_MODULES, lora_dropout=LORA_DROPOUT, bias="none", task_type="CAUSAL_LM")
model = get_peft_model(model, peft_config)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)")

Trainable: 23,592,960 / 1,544,985,600 (1.53%)


## Training

In [15]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR, num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS, learning_rate=LEARNING_RATE,
    logging_steps=LOGGING_STEPS, fp16=True, optim="paged_adamw_32bit",
    warmup_ratio=0.03, lr_scheduler_type="cosine",
    eval_strategy="steps", eval_steps=EVAL_STEPS,
    save_strategy="steps", save_steps=SAVE_STEPS, save_total_limit=3,
    load_best_model_at_end=True, metric_for_best_model="eval_loss", greater_is_better=False,
    push_to_hub=PUSH_TO_HUB, hub_model_id=HF_MODEL_NAME if PUSH_TO_HUB else None,
    hub_strategy="every_save", report_to=['tensorboard'], seed=42
)

trainer = Trainer(model=model, train_dataset=tok_train, eval_dataset=tok_eval, args=training_args,
                  data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False))

print("Starting training...")
trainer.train()

Starting training...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
50,1.447500,1.360001
100,1.156700,1.251805
150,1.178900,1.190388
200,1.132400,1.154461
250,1.215500,1.127931
300,1.135900,1.107312
350,1.149200,1.093216
400,1.096400,1.076012
450,1.097700,1.065769
500,1.080800,1.064373


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/pyt

TrainOutput(global_step=1689, training_loss=0.965253610876342, metrics={'train_runtime': 4518.3327, 'train_samples_per_second': 2.988, 'train_steps_per_second': 0.374, 'total_flos': 1.108216774656e+17, 'train_loss': 0.965253610876342, 'epoch': 3.0})

## Fine-tuned Model Evaluation (AFTER Training)

In [16]:
from transformers import logging
logging.set_verbosity(logging.CRITICAL)
model.eval()

ft_pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

print("Evaluating fine-tuned model on CROP-benchmark...")
ft_results = evaluate_mcq(ft_pipe, gen_config, english_benchmark)
print(f"\n📊 Fine-tuned Accuracy: {ft_results['accuracy']:.2%} ({ft_results['correct']}/{ft_results['total']})")

Evaluating fine-tuned model on CROP-benchmark...

📊 Fine-tuned Accuracy: 82.20% (411/500)


## Evaluation Summary

In [17]:
# Perplexity
def calc_ppl(model, tokenizer, texts):
    model.eval()
    total_loss, total_tokens = 0, 0
    with torch.no_grad():
        for text in texts[:100]:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
            out = model(**inputs, labels=inputs["input_ids"])
            total_loss += out.loss.item() * inputs["input_ids"].size(1)
            total_tokens += inputs["input_ids"].size(1)
    return math.exp(total_loss / total_tokens)

perplexity = calc_ppl(model, tokenizer, [s["text"] for s in eval_ds])

# Get losses
history = trainer.state.log_history
train_losses = [(h['step'], h['loss']) for h in history if 'loss' in h and 'eval_loss' not in h]
eval_losses = [(h['step'], h['eval_loss']) for h in history if 'eval_loss' in h]

print("\n" + "="*60)
print("EVALUATION SUMMARY")
print("="*60)
print(f"\n📉 Training Loss: {train_losses[0][1]:.4f} → {train_losses[-1][1]:.4f}")
print(f"📈 Validation Loss: {eval_losses[0][1]:.4f} → {min(e[1] for e in eval_losses):.4f}")
print(f"🎯 Perplexity: {perplexity:.2f}")
print(f"\n📋 CROP-benchmark Results:")
print(f"   Base Model:      {base_results['accuracy']:.2%}")
print(f"   Fine-tuned:      {ft_results['accuracy']:.2%}")
print(f"   Improvement:     +{(ft_results['accuracy']-base_results['accuracy'])*100:.1f}%")

print(f"\n📊 By Difficulty:")
for level, stats in sorted(ft_results['by_level'].items()):
    acc = stats['correct']/stats['total'] if stats['total'] else 0
    print(f"   Level {level}: {acc:.2%} ({stats['correct']}/{stats['total']})")


EVALUATION SUMMARY

📉 Training Loss: 2.0458 → 0.7141
📈 Validation Loss: 1.3600 → 0.9917
🎯 Perplexity: 3.12

📋 CROP-benchmark Results:
   Base Model:      73.20%
   Fine-tuned:      82.20%
   Improvement:     +9.0%

📊 By Difficulty:
   Level 0: 28.07% (16/57)
   Level 1: 85.41% (240/281)
   Level 2: 95.68% (155/162)


## Save & Push to HuggingFace

In [20]:
# Save results
os.makedirs(OUTPUT_DIR, exist_ok=True)
results = {
    "benchmark": "CROP-benchmark",
    "base_accuracy": base_results['accuracy'],
    "finetuned_accuracy": ft_results['accuracy'],
    "perplexity": perplexity,
    "best_eval_loss": min(e[1] for e in eval_losses),
    "accuracy_by_level": {k: v['correct']/v['total'] for k,v in ft_results['by_level'].items() if v['total']>0}
}
with open(f"{OUTPUT_DIR}/evaluation_results.json", "w") as f:
    json.dump(results, f, indent=2)

if PUSH_TO_HUB:
    from huggingface_hub import HfApi

    # Model card
    card = f"""---
language: [en]
license: apache-2.0
tags: [agriculture, phi-2, qlora, crop-science]
datasets: [AI4Agr/CROP-dataset]
base_model: microsoft/phi-2
model-index:
- name: AgricGPT-Phi2
  results:
  - task: {{type: question-answering, name: Agricultural MCQ}}
    dataset: {{name: CROP-benchmark, type: AI4Agr/CROP-benchmark}}
    metrics:
    - {{type: accuracy, value: {ft_results['accuracy']*100:.1f}, name: Accuracy}}
---

# AgricGPT - Agricultural QA Model

## Benchmark Results (CROP-benchmark)

| Model | Accuracy |
|-------|----------|
| Base Phi-2 | {base_results['accuracy']*100:.1f}% |
| **AgricGPT** | **{ft_results['accuracy']*100:.1f}%** (+{(ft_results['accuracy']-base_results['accuracy'])*100:.1f}%) |

| Metric | Value |
|--------|-------|
| Perplexity | {perplexity:.2f} |
| Val Loss | {results['best_eval_loss']:.4f} |
"""
    with open(f"{OUTPUT_DIR}/README.md", "w") as f:
        f.write(card)

    trainer.push_to_hub()
    api = HfApi()
    username = api.whoami()['name']
    api.upload_file(
        path_or_fileobj=f"{OUTPUT_DIR}/README.md",
        path_in_repo="README.md",
        repo_id=f"{username}/{HF_MODEL_NAME}",
        repo_type="model"
    )
    print(f"\nhttps://huggingface.co/{api.whoami()['name']}/{HF_MODEL_NAME}")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...results/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...45252.825cfec849d9.4778.0: 100%|##########| 6.41kB / 6.41kB            

  ...45462.825cfec849d9.4778.1: 100%|##########| 50.3kB / 50.3kB            

  ...adapter_model.safetensors:  44%|####4     | 41.9MB / 94.4MB            

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.



✅ Done! https://huggingface.co/Ajegetina/agricgpt-v1-phi2
